In [132]:
import pandas as pd

df = pd.read_csv("../data/scraped_data.csv")
df.head()

,Unnamed: 0,addresses,titles,areas,bedrooms,bathrooms,parkingSpots,prices
0,0,"Rua das Patativas, 5 - Imbuí, Salvador - BA","Apartamento com 2 Quartos à Venda, 66m²",66,2,2,2,R$ 460.000
1,1,"Rua Padre Silveira Sales, 04 - Jaguaribe, Salv...",OCEAN HOUSE - 42m²,42,1,1,1,NaN
2,2,"Rua das Pitangas de São Tomé - Sao Tome, Salva...","Casa com 5 Quartos à Venda, 324m²",324,5,3,4,R$ 340.000
3,3,"1ª Travessa Trasybulo Feeraz, 19 - Cidade Nova...","Casa com 2 Quartos à Venda, 76m²",76,2,2,1,R$ 50.000
4,4,"Ladeira do Jacaré - Macaubas, Salvador - BA","Casa com 3 Quartos à Venda, 90m²",90,3,2,1,R$ 195.000


In [133]:
df = df.drop_duplicates()
df = df.drop(df.columns[0], axis=1)
df = df.dropna(subset=["prices"])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 227 entries, 0 to 287
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   addresses     227 non-null    object
 1   titles        227 non-null    object
 2   areas         227 non-null    object
 3   bedrooms      227 non-null    object
 4   bathrooms     227 non-null    object
 5   parkingSpots  227 non-null    object
 6   prices        227 non-null    object
dtypes: object(7)
memory usage: 14.2+ KB


In [134]:
df['type'] = ''
# Drop offices, rooms and other possible non-residential titles
df = df[~df['titles'].str.contains('sala', case=False)]

df.loc[df['titles'].str.contains('casa', case=False), 'type'] = 'house'
df.loc[df['titles'].str.contains('apartamento|residencial', case=False), 'type'] = 'apartment'

df = df.drop("titles", axis=1)

df.head()

,addresses,areas,bedrooms,bathrooms,parkingSpots,prices,type
0,"Rua das Patativas, 5 - Imbuí, Salvador - BA",66,2,2,2,R$ 460.000,apartment
2,"Rua das Pitangas de São Tomé - Sao Tome, Salva...",324,5,3,4,R$ 340.000,house
3,"1ª Travessa Trasybulo Feeraz, 19 - Cidade Nova...",76,2,2,1,R$ 50.000,house
4,"Ladeira do Jacaré - Macaubas, Salvador - BA",90,3,2,1,R$ 195.000,house
5,"Rua Conde de Porto Alegre, 260 - Iapi, Salvado...",56,2,1,--,R$ 140.000,apartment


In [135]:
addresses = df.addresses.str.split(" - ", expand=True)
addresses = addresses.rename(columns={0: "streets", 1: "neighborhood"})
addresses.neighborhood = addresses.neighborhood.str.replace(", Salvador", "")
addresses = addresses.drop(addresses.columns[2], axis=1)

addresses

,streets,neighborhood
0,"Rua das Patativas, 5",Imbuí
2,Rua das Pitangas de São Tomé,Sao Tome
3,"1ª Travessa Trasybulo Feeraz, 19",Cidade Nova
4,Ladeira do Jacaré,Macaubas
5,"Rua Conde de Porto Alegre, 260",Iapi
...,...,...
283,"Rua Rubens Zardival, 12",Sussuarana
284,"Avenida Jorge Amado, 214",Imbuí
285,"Parque Bela Vista, Salvador",BA
286,"Largo do Papagaio, 0",Ribeira


In [136]:
df = df.drop("addresses", axis=1)
df = pd.concat([df, addresses], axis=1)
df.head()

,areas,bedrooms,bathrooms,parkingSpots,prices,type,streets,neighborhood
0,66,2,2,2,R$ 460.000,apartment,"Rua das Patativas, 5",Imbuí
2,324,5,3,4,R$ 340.000,house,Rua das Pitangas de São Tomé,Sao Tome
3,76,2,2,1,R$ 50.000,house,"1ª Travessa Trasybulo Feeraz, 19",Cidade Nova
4,90,3,2,1,R$ 195.000,house,Ladeira do Jacaré,Macaubas
5,56,2,1,--,R$ 140.000,apartment,"Rua Conde de Porto Alegre, 260",Iapi


In [137]:
df.prices = df.prices.str.strip(" ")
df.prices = df.prices.str.replace("R$ ", "")
df.prices = df.prices.str.replace("      Preço abaixo do mercado", "")
df.prices = df.prices.str.replace("A partir de     ", "")
df.prices = df.prices.str.replace(".", "")

df.prices = pd.to_numeric(df.prices)

df.head()

,areas,bedrooms,bathrooms,parkingSpots,prices,type,streets,neighborhood
0,66,2,2,2,460000,apartment,"Rua das Patativas, 5",Imbuí
2,324,5,3,4,340000,house,Rua das Pitangas de São Tomé,Sao Tome
3,76,2,2,1,50000,house,"1ª Travessa Trasybulo Feeraz, 19",Cidade Nova
4,90,3,2,1,195000,house,Ladeira do Jacaré,Macaubas
5,56,2,1,--,140000,apartment,"Rua Conde de Porto Alegre, 260",Iapi


In [138]:
df.replace(" -- ", "", inplace=True)

def parse_range_and_mean(value):
    if '-' in value:
        start, end = map(int, value.split('-'))
        return (start + end) / 2
    else:
        return float(value) if value != '' else None  # Handle empty strings

# Apply the function to the specified columns
columns_to_transform = ['areas', 'bedrooms', 'bathrooms', 'parkingSpots']  # Replace with your column names
for column in columns_to_transform:
    df[column] = df[column].apply(parse_range_and_mean)

df.head()

,areas,bedrooms,bathrooms,parkingSpots,prices,type,streets,neighborhood
0,66.0,2.0,2.0,2.0,460000,apartment,"Rua das Patativas, 5",Imbuí
2,324.0,5.0,3.0,4.0,340000,house,Rua das Pitangas de São Tomé,Sao Tome
3,76.0,2.0,2.0,1.0,50000,house,"1ª Travessa Trasybulo Feeraz, 19",Cidade Nova
4,90.0,3.0,2.0,1.0,195000,house,Ladeira do Jacaré,Macaubas
5,56.0,2.0,1.0,NaN,140000,apartment,"Rua Conde de Porto Alegre, 260",Iapi


In [139]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 226 entries, 0 to 287
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   areas         226 non-null    float64
 1   bedrooms      224 non-null    float64
 2   bathrooms     224 non-null    float64
 3   parkingSpots  183 non-null    float64
 4   prices        226 non-null    int64  
 5   type          226 non-null    object 
 6   streets       226 non-null    object 
 7   neighborhood  226 non-null    object 
dtypes: float64(4), int64(1), object(3)
memory usage: 15.9+ KB
